# Clustering Crypto

In [26]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import numpy as np
from path import Path
%matplotlib inline


### Fetching Cryptocurrency Data

In [27]:
# Use the following endpoint to fetch json data
# url = "https://min-api.cryptocompare.com/data/all/coinlist" #using the alternative file

In [28]:
# Create a DataFrame 
file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path)
crypto_df


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


### Data Preprocessing

In [29]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.set_index("Unnamed: 0")
crypto_df.index.name = None

In [30]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [31]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [32]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(columns = 'IsTrading')
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [33]:
# Remove rows with at least 1 null value
crypto_df= crypto_df.dropna()

In [34]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [35]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[crypto_df!='N/A']
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [36]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
c_coin = pd.DataFrame(crypto_df["CoinName"], index= crypto_df.index)
c_coin

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [37]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
c_name = crypto_df.drop(columns = 'CoinName')
c_name.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [38]:
# Create dummy variables for text features
v_dummy= pd.get_dummies(c_name, columns=['Algorithm', 'ProofType'])
v_dummy.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
# Standardize data
scaled_data = StandardScaler().fit_transform(v_dummy)
scaled_data

array([[-0.11674788, -0.15286468, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.09358885, -0.14499604, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231,  4.4937636 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.13215444, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.15284989, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

### Reducing Dimensions Using PCA

In [40]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(scaled_data)

In [41]:
# Create a DataFrame with the principal components data
pcd_data = pd.DataFrame(crypto_pca, columns=["PC1", "PC2", "PC3"],index= crypto_df.index)
pcd_data

,PC1,PC2,PC3
42,-0.335090,0.979115,-0.625119
404,-0.318401,0.979266,-0.625571
1337,2.310073,1.662786,-0.713740
BTC,-0.149973,-1.317589,0.214553
ETH,-0.148492,-2.023688,0.460592
...,...,...,...
ZEPH,2.462768,0.798475,-0.117083
GAP,-0.333132,0.979003,-0.625136
BDX,0.328836,-2.291657,0.469444
ZEN,-0.158459,-1.988697,0.258715


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [42]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(pcd_data)
    inertia.append(k_model.inertia_)
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Create the Elbow Curve using hvPlot
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

c:\Users\karen\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  f"KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [43]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=1)

# Fit the model
model.fit(pcd_data)

# Predict clusters
pre_clusters= model.predict(pcd_data)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcd_data["class"] = model.labels_
pcd_data

,PC1,PC2,PC3,class
42,-0.335090,0.979115,-0.625119,1
404,-0.318401,0.979266,-0.625571,1
1337,2.310073,1.662786,-0.713740,4
BTC,-0.149973,-1.317589,0.214553,0
ETH,-0.148492,-2.023688,0.460592,0
...,...,...,...,...
ZEPH,2.462768,0.798475,-0.117083,4
GAP,-0.333132,0.979003,-0.625136,1
BDX,0.328836,-2.291657,0.469444,0
ZEN,-0.158459,-1.988697,0.258715,0


In [44]:
clusters_df = pd.concat([crypto_df.drop(columns="CoinName"),c_coin, pcd_data], axis=1)
clusters_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,PC1,PC2,PC3,class
42,Scrypt,PoW/PoS,4.199995e+01,42,42 Coin,-0.335090,0.979115,-0.625119,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,404Coin,-0.318401,0.979266,-0.625571,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,EliteCoin,2.310073,1.662786,-0.713740,4
BTC,SHA-256,PoW,1.792718e+07,21000000,Bitcoin,-0.149973,-1.317589,0.214553,0
ETH,Ethash,PoW,1.076842e+08,0,Ethereum,-0.148492,-2.023688,0.460592,0
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,ZEPHYR,2.462768,0.798475,-0.117083,4
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,Gapcoin,-0.333132,0.979003,-0.625136,1
BDX,CryptoNight,PoW,9.802226e+08,1400222610,Beldex,0.328836,-2.291657,0.469444,0
ZEN,Equihash,PoW,7.296538e+06,21000000,Horizen,-0.158459,-1.988697,0.258715,0


In [45]:
clusters_df["CoinName"]

42          42 Coin
404         404Coin
1337      EliteCoin
BTC         Bitcoin
ETH        Ethereum
           ...     
ZEPH         ZEPHYR
GAP         Gapcoin
BDX          Beldex
ZEN         Horizen
XBC     BitcoinPlus
Name: CoinName, Length: 533, dtype: object

### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [46]:
# Scale data to create the scatter plot
scaler = MinMaxScaler()

rescale_columns = ["TotalCoinsMined", "TotalCoinSupply"]
x = clusters_df[rescale_columns].values
x_scaled = scaler.fit_transform(x)

x_scaled

array([[5.94230127e-03, 4.20000000e-11],
       [7.00182308e-03, 5.32000000e-04],
       [3.53420682e-02, 3.14159265e-01],
       ...,
       [6.92655266e-03, 1.40022261e-03],
       [5.94962775e-03, 2.10000000e-05],
       [5.94243008e-03, 1.00000000e-06]])

In [47]:
scaled_df = pd.DataFrame(x_scaled, columns=rescale_columns, index=crypto_df.index)
scaled_df

,TotalCoinsMined,TotalCoinSupply
42,0.005942,4.200000e-11
404,0.007002,5.320000e-04
1337,0.035342,3.141593e-01
BTC,0.005960,2.100000e-05
ETH,0.006050,0.000000e+00
...,...,...
ZEPH,0.007951,2.000000e-03
GAP,0.005957,2.500000e-04
BDX,0.006927,1.400223e-03
ZEN,0.005950,2.100000e-05


In [48]:
scaled_df["class"] = clusters_df["class"]
scaled_df["CoinName"] = clusters_df["CoinName"]

In [49]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
plot = scaled_df.hvplot(
    x="TotalCoinsMined", 
    y="TotalCoinSupply",
    c = "class",
    hover_cols=['CoinName'],
).opts(yformatter='%.0f')

#### Table of Tradable Cryptocurrencies

In [50]:
# Table with tradable cryptos
clusters_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [51]:
# Print the total number of tradable cryptocurrencies
print(f"Total number of tradable cryptocurrencies is: {len(clusters_df)}")

Total number of tradable cryptocurrencies is: 533
